In [1]:
import sklearn

In [2]:
from sklearn.datasets import make_regression
from sklearn.linear_model import SGDRegressor

import pandas as pd
import numpy as np

In [3]:
X, y = make_regression()
pdf = pd.DataFrame(X)
pdf.columns = ['c{}'.format(x) for x in range(100)]

In [4]:
X.shape

(100, 100)

In [5]:
X1 = pdf[['c{}'.format(x) for x in range(50, 100)]]
X2 = pdf[['c{}'.format(x) for x in range(50)]]

In [12]:
type(pdf)

pandas.core.frame.DataFrame

In [15]:
bool_mask = np.ones(pdf.shape[1], dtype=np.bool)
bool_mask[0] = False

In [18]:
pdf[pdf.columns[bool_mask]].shape

(100, 99)

In [25]:
np.array(pdf)[:, bool_mask].shape

(100, 99)

In [37]:
np.array([1,2,3]).tolist() + [1,2,3]

[1, 2, 3, 1, 2, 3]

In [73]:
"""
To do:

penalise the incoming expanded feature size with `lambda_`.

Essentially if absolute value of incoming feature is less than `lambda_` 
then the coefficient is set to 0.
"""

class GraftingRegressor(SGDRegressor):
    def __init__(self, loss="squared_loss", penalty="l2", alpha=0.0001,
                 l1_ratio=0.15, fit_intercept=True, max_iter=None, tol=None,
                 shuffle=True, verbose=0, epsilon=0.1,
                 random_state=None, learning_rate="invscaling", eta0=0.01,
                 power_t=0.25, warm_start=False, average=False, n_iter=None, reg_penalty=None):
        super(GraftingRegressor, self).__init__(loss=loss, penalty=penalty,
                                           alpha=alpha, l1_ratio=l1_ratio,
                                           fit_intercept=fit_intercept,
                                           max_iter=max_iter, tol=tol,
                                           shuffle=shuffle,
                                           verbose=verbose,
                                           epsilon=epsilon,
                                           random_state=random_state,
                                           learning_rate=learning_rate,
                                           eta0=eta0, power_t=power_t,
                                           warm_start=warm_start,
                                           average=average, n_iter=n_iter)
        self.filter_cols = []
        self.base_shape = None
        self.reg_penalty = reg_penalty if reg_penalty is not None else l1_ratio
    
    def _fit_columns(self, X, return_x=True):
        """
        Method filter through "unselected" columns. The goal of this 
        method is to filter any uninformative columns.
        
        This will be selected based on index only?
        
        If return_x is false, it will only return the boolean mask.
        """
        import pandas
        bool_mask = np.ones((X.shape[1],), dtype=np.bool)
        if len(self.filter_cols) == 0:
            if return_x:
                return X
            else:
                return bool_mask
        # otherwise...
        bool_mask[self.filter_cols] = False
        if not return_x:
            return bool_mask
        if type(X) is pandas.core.frame.DataFrame:
            return X[X.columns[bool_mask]]
        else:
            return X[:, bool_mask]
    
    def _reg_penalty(self):
        bool_mask = np.zeros((self.coef_.shape[0],), dtype=np.bool)
        keep_cols = np.argwhere(np.abs(self.coef_) > self.reg_penalty)        
        mask = np.array(list(set(keep_cols.flatten().tolist() + list(range(self.base_shape)))))
        self.coef_ = self.coef_[mask]
        bool_mask[mask] = True
        self.filter_cols = np.argwhere(~bool_mask).flatten().tolist()        
    
    def _partial_grafting_fit(self, X, y):
        """
        Partial fit grafting method to expand the coefficient listing
        to taking into account new coefficients
        """
        # require to know the base shape to determine/
        # check for irrelevant columns in the future.
        self.base_shape = self.coef_.shape[0]
        
        X = self._fit_columns(X)
        n_samples, n_features = X.shape
        coef_list = np.zeros(n_features, dtype=np.float64, order="C")
        coef_list[:self.coef_.shape[0]] = self.coef_.copy()
        self.coef_ = coef_list.copy()
    
    def partial_fit(self, X, y, sample_weight=None):
        self._partial_grafting_fit(X, y)
        super(GraftingRegressor, self).partial_fit(X, y, sample_weight=None)  
        
        # update parameters based on weight of regularizer penalty
        self._reg_penalty()
        return self
    
    def predict(self, X):
        X = self._fit_columns(X)
        return super(GraftingRegressor, self).predict(X)        

In [74]:
model = GraftingRegressor(max_iter=1000)
model.fit(X1, y)

GraftingRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
         fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
         loss='squared_loss', max_iter=1000, n_iter=None, penalty='l2',
         power_t=0.25, random_state=None, reg_penalty=0.15, shuffle=True,
         tol=None, verbose=0, warm_start=False)

In [75]:
len(model.coef_)

50

In [76]:
model.partial_fit(pdf, y)

GraftingRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
         fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
         loss='squared_loss', max_iter=1000, n_iter=None, penalty='l2',
         power_t=0.25, random_state=None, reg_penalty=0.15, shuffle=True,
         tol=None, verbose=0, warm_start=False)

In [77]:
len(model.coef_)

97

In [78]:
model.predict(pdf)

array([-127.2228443 ,  -53.17851709,  121.10895529, -306.05969936,
       -186.75076119, -254.57819733,  -61.46087582,   53.53935409,
       -138.19323121,  -46.64270561,   25.65567076,  -28.34779821,
         -5.96198081, -247.83746434,  -47.43007955, -177.77763846,
       -363.15477731,    8.65385062, -143.06744995, -173.53884928,
       -316.14631877, -140.55663957,    9.70001943,   88.4572901 ,
       -270.23814726, -216.86739912,  161.67752747, -378.75797285,
       -382.03785141,   11.64379202,  335.44746081,  -90.81039223,
         28.78536154,    9.26124103,  232.67488726, -195.1287139 ,
        -10.06136878,  170.43023882,   39.79872617, -102.19041616,
        150.16168143,  -60.64117512, -289.0414291 ,  -63.78643728,
       -121.26107664,  240.73075773, -366.76921855, -404.37710068,
         16.98329469,  -16.77090811, -153.9061065 , -252.48765113,
        184.35296147,  -66.95808695,  -56.58789881,   95.59657726,
       -104.01863851,  136.53187281, -360.07737282, -181.88381